In [32]:
import pandas as pd

In [33]:
DATAPATH = 'data/tongdaxin/'
PATH = DATAPATH

def read_file_loc(file_name, splits):
    with open(file_name, 'r') as f:
        buf_lis = f.read().split('\n')
    return [x.split(splits) for x in buf_lis[:-1]]


In [43]:
buf_line = read_file_loc(PATH+'tdxhy.cfg', '|')
buf_lis = []
mapping = {'0': 'sz.', '1': 'sh.', '2': 'bj.'}
for x in buf_line:
    # x[1] = mapping[x[0]] + x[1]
    buf_lis.append(x)

df = pd.DataFrame(buf_lis, columns=[
                    'c0', 'code', 'block', 'c1', 'c2', 'c3'])
# print(df)
df.drop(df.columns[[0, 3, 4, 5]], axis=1, inplace=True)

df = df[(df['block'] != '')]
# df = df[df.code.str.startswith(('sz','sh'))]
df['block5'] = df['block'].str[0:5]
df

,code,block,block5
0,000001,T1001,T1001
1,000002,T110201,T1102
2,000004,T1205,T1205
3,000005,T0805,T0805
4,000006,T110202,T1102
...,...,...,...
5208,873223,T070402,T0704
5209,873305,T070402,T0704
5210,873339,T1204,T1204
5211,873527,T020401,T0204


In [44]:
# get_block_zs_tdx_loc
block="hy"
buf_line = read_file_loc(PATH+'tdxzs3.cfg', '|')
# "dq" 地区 --- 32 
# "hy" 行业 --- 56   
# "gn" 概念 --- 239  216 
# "fg" 板块指数 --- 132
# "yjhy" 研究行业 --- 
mapping = {'hy': '2', 'dq': '3', 'gn': '4',
            'fg': '5', 'yjhy': '12'}
mydf = pd.DataFrame(buf_line, columns=[
    'name', 'code', 'type', 't1', 't2', 'block'])
mydf



,name,code,type,t1,t2,block
0,轮动趋势,880081,5,2,0,轮动趋势
1,板块趋势,880082,5,2,0,板块趋势
2,黑龙江,880201,3,1,0,1
3,新疆板块,880202,3,1,0,2
4,吉林板块,880203,3,1,0,3
...,...,...,...,...,...,...
1008,固废治理,881474,12,1,1,X630203
1009,生态修复,881475,12,1,1,X630204
1010,环境监测,881476,12,1,1,X6303
1011,综合,881477,12,1,0,X99


In [53]:
mydf['type'].value_counts()

12    465
4     239
2     145
5     132
3      32
Name: type, dtype: int64

In [38]:
mydf[mydf['code']=="881326"]

,name,code,type,t1,t2,block
864,消费电子,881326,12,1,0,X4002


In [61]:
block = "gn"
dg = mydf.groupby(by='type')
# df.to_excel('block.xlsx')
# if (block == 'zs'):
#     return df
# temp = dg.reset_index(drop=True)
temp = dg.get_group(mapping[block]).reset_index(drop=True)
# temp.drop(temp.columns[[2, 3, 4]], axis=1, inplace=True)
temp


,name,code,type,t1,t2,block,block5,num,stocks
0,含H股,880501,4,1,0,[],含H股,0,
1,含B股,880502,4,1,0,[],含B股,0,
2,稀缺资源,880505,4,1,0,[],稀缺资源,0,
3,5G概念,880506,4,2,0,[],5G概念,0,
4,国防军工,880507,4,1,0,[],国防军工,0,
...,...,...,...,...,...,...,...,...,...
234,医美概念,880973,4,2,0,[],医美概念,0,
235,烟草概念,880974,4,2,0,[],烟草概念,0,
236,有机硅概念,880975,4,2,0,[],有机硅,0,
237,新冠检测,880976,4,2,0,[],新冠检测,0,


In [39]:
df.drop(df.columns[[2, 3, 4]], axis=1, inplace=True)


In [45]:
stocklist =df
blocklist =mydf


In [47]:
blocklist


,name,code,type,t1,t2,block
0,轮动趋势,880081,5,2,0,轮动趋势
1,板块趋势,880082,5,2,0,板块趋势
2,黑龙江,880201,3,1,0,1
3,新疆板块,880202,3,1,0,2
4,吉林板块,880203,3,1,0,3
...,...,...,...,...,...,...
1008,固废治理,881474,12,1,1,X630203
1009,生态修复,881475,12,1,1,X630204
1010,环境监测,881476,12,1,1,X6303
1011,综合,881477,12,1,0,X99


In [48]:
blocklist['block5'] = blocklist['block'].str[0:5]
 # print(blocklist)
blocklist['num'] = 0
blocklist['stocks'] = ''



In [49]:
blocklist[blocklist['code']=="881326"]


,name,code,type,t1,t2,block,block5,num,stocks
864,消费电子,881326,12,1,0,X4002,X4002,0,


In [50]:
for i in range(len(blocklist)):
    blockkey = blocklist.iat[i, 2]
    if (len(blockkey) == 5):
        datai = stocklist[stocklist['block5'] == blockkey]  # ���ݰ�����ƹ���
    else:
        datai = stocklist[stocklist['block'] == blockkey]  # ���ݰ�����ƹ���
    # ����ڽ������������
    datai = datai.sort_values(by=['code'], ascending=[True])
    # datai.reset_index(drop=True, inplace=True)
    codelist = datai['code'].tolist()

    blocklist.iat[i, 4] = len(codelist)
    blocklist.iat[i, 5] = str(codelist)
blocklist = blocklist.drop(blocklist[blocklist['num'] == 0].index)
blocklist


,name,code,type,t1,t2,block,block5,num,stocks


In [51]:
blocklist.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    0 non-null      object
 1   code    0 non-null      object
 2   type    0 non-null      object
 3   t1      0 non-null      object
 4   t2      0 non-null      object
 5   block   0 non-null      object
 6   block5  0 non-null      object
 7   num     0 non-null      int64 
 8   stocks  0 non-null      object
dtypes: int64(1), object(8)
memory usage: 124.0+ bytes


In [62]:
hy_df=pd.read_csv("hyblock.csv")
hy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    132 non-null    object
 1   code    132 non-null    int64 
 2   block   132 non-null    object
 3   block5  132 non-null    object
 4   num     132 non-null    int64 
 5   stocks  132 non-null    object
dtypes: int64(2), object(4)
memory usage: 6.3+ KB


In [63]:
hy_df.head()

,name,code,block,block5,num,stocks
0,煤炭,880301,T0101,T0101,35,"['000552', '000571', '000723', '000937', '0009..."
1,煤炭开采,880302,T010101,T0101,27,"['000552', '000571', '000937', '000983', '0021..."
2,焦炭加工,880303,T010102,T0101,8,"['000723', '600408', '600725', '600740', '6007..."
3,电力,880305,T0102,T0102,72,"['000027', '000037', '000040', '000155', '0005..."
4,水力发电,880306,T010201,T0102,18,"['000601', '000791', '000993', '002039', '6000..."


In [64]:
out_df = hy_df[['name', 'code']]
out_df.columns = ['板块名称','板块代码']
out_df


,板块名称,板块代码
0,煤炭,880301
1,煤炭开采,880302
2,焦炭加工,880303
3,电力,880305
4,水力发电,880306
...,...,...
127,半导体,880491
128,元器件,880492
129,软件服务,880493
130,互联网,880494


In [65]:
out_df.isnull().sum()


板块名称    0
板块代码    0
dtype: int64

In [66]:
out_df.to_csv("通达信板块.csv",index=False)

In [67]:
gn_df = pd.read_csv("gnblock.csv")
gn_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    215 non-null    object
 1   code    215 non-null    int64 
 2   tp      215 non-null    object
 3   num     215 non-null    int64 
 4   stocks  215 non-null    object
dtypes: int64(2), object(3)
memory usage: 8.5+ KB


In [68]:
out_gn_df = gn_df[['name', 'code']]
out_gn_df.columns = ['板块名称', '板块代码']
out_gn_df


,板块名称,板块代码
0,含H股,880501
1,含B股,880502
2,稀缺资源,880505
3,5G概念,880506
4,国防军工,880507
...,...,...
210,降解塑料,880971
211,医美概念,880973
212,烟草概念,880974
213,新冠检测,880976


In [69]:
out_gn_df.to_csv("通达信概念.csv", index=False)


In [ ]:
["股票代码","股票名称","板块名称"]

## 输出结果

In [85]:
mydf=hy_df[['name','stocks']]
mydf["stocks"] = mydf["stocks"].apply(eval)
mydf=mydf.explode('stocks')
# mydf.iloc[0]['stocks']
mydf.columns=['板块名称',"股票代码"]
mydf = mydf.reindex(columns=["股票代码", '板块名称'])
mydf.to_csv("通达信_板块名称_股票对应.csv",index=False)


/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_74203/4275946665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydf["stocks"] = mydf["stocks"].apply(eval)


In [84]:
mydf = gn_df[['name', 'stocks']]
mydf["stocks"] = mydf["stocks"].apply(eval)
mydf = mydf.explode('stocks')
# mydf.iloc[0]['stocks']
mydf.columns = ['板块名称', "股票代码"]
# 使用reindex()函数重新排列列
mydf = mydf.reindex(columns=["股票代码", '板块名称'])
mydf
mydf.to_csv("通达信_概念名称_股票对应.csv", index=False)


/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_74203/1218701606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydf["stocks"] = mydf["stocks"].apply(eval)


In [71]:
# hy_df  # 使用explode()函数拆分列表字段
hy_df_exploded = hy_df.explode("stocks")
out_hy_df_exploded = hy_df_exploded['name','stocks']
out_hy_df_exploded.columns=["板块名称",	"股票名称"]
out_hy_df_exploded


KeyError: ('name', 'stocks')

In [73]:
import pandas as pd

# 创建示例DataFrame
df = pd.DataFrame({
    "id": [1, 2],
    "name": ["Alice", "Bob"],
    "hobbies": [["reading", "hiking", "cooking"], ["swimming", "gardening"]]
})

# 打印原始DataFrame
print("Original DataFrame:")
print(df)
print(df.info())
# 使用explode()函数拆分列表字段
df_exploded = df.explode("hobbies")

# 打印拆分后的DataFrame
print("Exploded DataFrame:")
print(df_exploded)


Original DataFrame:
   id   name                     hobbies
0   1  Alice  [reading, hiking, cooking]
1   2    Bob       [swimming, gardening]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2 non-null      int64 
 1   name     2 non-null      object
 2   hobbies  2 non-null      object
dtypes: int64(1), object(2)
memory usage: 176.0+ bytes
None
Exploded DataFrame:
   id   name    hobbies
0   1  Alice    reading
0   1  Alice     hiking
0   1  Alice    cooking
1   2    Bob   swimming
1   2    Bob  gardening
